# Using the xID service from OCLC
Below I grab some records from a marcxml file, grab the ISSNs, and use the xISSN web service (to be discontinued in March 2016) to populate a dataframe with related ISSNs for the given ISSN.

In [1]:
import os
import numpy as np
import pandas as pd
#import glob
import os
#import codecs
import json
#import time
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus
import pymarc
import marcx
import io
os.chdir('/Volumes/jwa_drive1/git/issn')


## Let's get some records
I generated a managed set in Alma and exported the bib records in MARCXML format.
Below is a routine to parse the marc records and create a dataframe that holds the relevant fields from the marc records.

In [2]:
f = '/Volumes/jwa_drive1/git/issn/marcRecords.xml'
records = pymarc.parse_xml_to_array(io.open(f,mode='r',encoding='utf-8'))
columns = ['issn', 'mmsid', 'title','other_issns']
df = pd.DataFrame()

for rec in records:
    d = {}
    rec = marcx.FatRecord.from_record(rec)
    try:
        d['issn'] = rec['022']['a']
    except Exception as e:
        continue
        #d['issn'] = ''
    d['title'] = rec.title().replace('/','')
    d['mmsid'] = rec['001'].data
    d['other_issns'] = ''
    df = df.append(d,ignore_index=True)
## uncomment line below to verify that the records were loaded properly
#df.head()

In [6]:
base_url = 'http://xissn.worldcat.org/webservices/xid/issn/'
end_url = '?method=getForms&format=json'
# iterate through a slice of the dataframe.
# for each row, we grab the issn and use that as part of the query string to 
# send to the xISSN service. 
# the response is a string (bytes) that we convert to json.
# we navigate the json response to grab the list of issn numbers which we call 'ISSNs'
# ISSNs includes format information as well. we parse that list to extract only the desired
# issn number which we add to a newly constructed 'issn_list'
# we add the issn_list to the dataframe
for index,row in df[:20].iterrows():
    issn = row['issn']
    try:
        response = json.loads(urlopen(base_url + issn + end_url).read().decode('utf-8'))
    except Exception as e:
        print(index,e)
        continue
    ISSNs = response['group'][0]['list']
    issn_list = []
    for issn in ISSNs:
        issn_list.append(issn['issn'])
    #print(issn_list)
    row['other_issns'] = issn_list
        

14 Can't convert 'NoneType' object to str implicitly


In [7]:
df.head(20)

,issn,mmsid,other_issns,title
0,1043-9951,99174914290001161,[1043-9951],Women's health perspectives.
1,1562-8140,99174977580001161,[1562-8140],Newsletter
2,0424-2017,99175014040001161,[0424-2017],Scientific publications from Eastman Kodak Lab...
3,0730-7349,99174913540001161,[0730-7349],The chronicle of the Catholic Church in Lithua...
4,0079-7308,99174926670001161,[0079-7308],The Psychoanalytic study of the child.
5,0097-8043,99174960890001161,[0097-8043],Restaurant business.
6,0896-9205,99174720230001161,"[0896-9205, 1569-1632]",Critical sociology.
7,0072-9906,99175051470001161,[0072-9906],Handbook of sensory physiology.
8,0020-7152,99174820960001161,"[1568-5187, 0538-8082, 1745-2554, 0020-7152]",International journal of comparative sociology.
9,1045-0467,99175057040001161,"[0271-3624, 1045-0467]",Annual report of the president and treasurer
